## <a id = '0'> Índice </a>

* [**Entorno**](#1)  
   * [Librerías](#1d1)  
   * [Funciones](#1d2)  
   * [Constantes](#1d3)

* [**Lectura de datos**](#2)


## <a id = '1'> Entorno </a>
[índice](#0)

### <a id = '1d1'> Librerías </a>

In [1]:
import os
import pandas as pd
import numpy as np

from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox

from statsmodels.tsa.arima.model import ARIMA

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt

import math
import itertools

In [2]:
os.chdir("../")

### <a id = '1d2'> Funciones </a>

In [3]:
from src.utils import *

### <a id = '1d3'> Constantes </a>

In [13]:
from config.cons import data_folder, window_jason, cara_lateral, cara_anterior, cara_interior, cara_sepal, list_signals

## <a id = '2'> Lectura de datos </a>
[índice](#0)

In [5]:
proyect_path = os.getcwd()
data_path = proyect_path + data_folder

In [6]:
lateral_combinaciones = list(itertools.combinations(cara_lateral, 2))
anterior_combinaciones = list(itertools.combinations(cara_anterior, 2))
interior_combinaciones = list(itertools.combinations(cara_interior, 2))
sepal_combinaciones = list(itertools.combinations(cara_sepal, 2))

In [7]:
df_all = lectura_carpetas_dict(data_path)

In [8]:
df_mi = df_all["mi"]
df_sttc_mi = df_all["sttc_mi"]
df_sttc = df_all["sttc"]
df_other = df_all["other"]

In [12]:
#Lectura de windows
import json
with open(window_jason, "r") as file:
    dict_data_jump = json.load(file)  

In [ ]:
df_mi_smooth_part = dict_apply_smooth(df_mi, list_signals, dict_window = dict_data_jump["MI"])
df_sttc_mi_smooth_part = dict_apply_smooth(df_sttc_mi, list_signals, dict_window = dict_data_jump['STTC MI'])
df_sttc_smooth_part = dict_apply_smooth(df_sttc, list_signals, dict_window = dict_data_jump['STTC'])
df_other_smooth_part = dict_apply_smooth(df_other, list_signals, dict_window = dict_data_jump['OTHER'])

In [76]:
def get_df_acf_pacf(df, list_signals):
    acf_pacf_dict = {}
    patient_id = df["patient_id"].values[0]
    label = df["label"].values[0]

    for signal in list_signals:
        
        serie_acf, serie_pacf = acf_pacf(df[signal], lags = 5)
        acf_pacf_dict["acf_" + signal] = (pd.DataFrame(
                    [serie_acf[1:6]], 
                    columns=["acf_" + signal + "_lag_" + str(i) for i in range(1,len(serie_acf[1:6])+1)]))
        acf_pacf_dict["pacf_" + signal] = (pd.DataFrame(
                    [serie_pacf[1:6]], 
                    columns=["pacf_" + signal + "_lag_" + str(i) for i in range(1,len(serie_pacf[1:6])+1)]))

    
    result_df = pd.concat(acf_pacf_dict.values(), axis=1)
    result_df["patient_id"] = patient_id
    result_df["label"] = label

    return result_df

def genera_df_acf_pacf(df, list_signals):
    dict_acf_pacf = {
        pacient: get_df_acf_pacf(df[pacient], list_signals)
        for pacient in df.keys()}
    return dict_acf_pacf

def dict_to_dataframe(dict):
    return pd.concat(dict.values(), axis=0, ignore_index=True)
    

In [77]:
df_mi_acf_pacf = dict_to_dataframe(genera_df_acf_pacf(df_mi, list_signals))

In [78]:
df_mi_acf_pacf

,acf_AVL_lag_1,acf_AVL_lag_2,acf_AVL_lag_3,acf_AVL_lag_4,acf_AVL_lag_5,pacf_AVL_lag_1,pacf_AVL_lag_2,pacf_AVL_lag_3,pacf_AVL_lag_4,pacf_AVL_lag_5,...,acf_I_lag_3,acf_I_lag_4,acf_I_lag_5,pacf_I_lag_1,pacf_I_lag_2,pacf_I_lag_3,pacf_I_lag_4,pacf_I_lag_5,patient_id,label
0,0.735214,0.257987,-0.026348,-0.070716,-0.021364,0.735950,-0.617651,0.424609,-0.235623,0.124187,...,-0.030533,-0.084507,-0.044453,0.748239,-0.656654,0.499216,-0.345425,0.163117,patient_2415,mi
1,0.973577,0.917267,0.851673,0.788011,0.735943,0.974551,-0.609832,0.209014,0.010145,0.198126,...,0.778006,0.680650,0.598779,0.958874,-0.551249,0.131271,-0.117393,0.252021,patient_3086,mi
2,0.949951,0.849644,0.736250,0.618051,0.505658,0.950902,-0.551931,0.104146,-0.151717,0.072800,...,0.548454,0.422314,0.325156,0.902589,-0.547207,0.369367,-0.272692,0.214318,patient_2171,mi
3,0.869464,0.582898,0.289777,0.071885,-0.049304,0.870335,-0.715068,0.347406,-0.150766,0.084550,...,0.341806,0.105193,-0.045003,0.887937,-0.747131,0.303107,-0.066461,0.040622,patient_1450,mi
4,0.831092,0.494637,0.233241,0.096976,0.025254,0.831924,-0.638093,0.489554,-0.395428,0.235321,...,0.328256,0.230415,0.168351,0.837361,-0.557402,0.479678,-0.348755,0.250243,patient_944,mi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.901574,0.669546,0.399892,0.168654,0.018030,0.902477,-0.773845,0.342310,-0.017291,0.073791,...,0.494545,0.275856,0.096209,0.918710,-0.763170,0.349925,-0.284167,0.235637,patient_1893,mi
596,0.974347,0.919619,0.854892,0.790058,0.740139,0.975323,-0.611171,0.181956,-0.012764,0.384584,...,0.734436,0.621127,0.543536,0.952022,-0.588973,0.159262,-0.084784,0.479166,patient_3048,mi
597,0.921303,0.772958,0.643946,0.557590,0.502374,0.922225,-0.508312,0.336610,-0.032606,0.055510,...,0.363546,0.274495,0.231616,0.854353,-0.593497,0.549889,-0.369704,0.276397,patient_76,mi
598,0.832513,0.514123,0.291312,0.191547,0.149944,0.833346,-0.586886,0.499341,-0.354636,0.325491,...,0.319943,0.199873,0.141776,0.835878,-0.540270,0.386867,-0.256158,0.249071,patient_550,mi


In [74]:
temp_df_final.shape

(600, 122)